In [1]:
import pandas as pd 
import jsonlines
import json
from bs4 import BeautifulSoup
def list_of_discourse_sents_to_df(row):
    tuples = list(zip(row['sentences'], row['discourse_preds']))
    dicts = list(map(lambda x: {'sentence': x[0], 'discourse': x[1]}, tuples))
    return pd.DataFrame(dicts)

def match_archive_articles(doc_id):
    article = (
        beat_articles_discourse_labeled
            .loc[lambda df: df['suid'] == doc_id]
            .iloc[0]
    )
    article_tickers = article['security_figi']
    article_tickers = set(list(filter(lambda x: x is not None, article_tickers)))
    
    article_date = article['timeofarrival']
    
    archive = (
        news_archive_df
            .loc[lambda df: df['timeofarrival'] < article_date]
            .loc[lambda df: df['security_figi'].apply(lambda x: len(x & article_tickers) > 0)]
    )
    return article, archive

def parse_body(body):
    try:
        text = BeautifulSoup(body).get_text(' ')
    except:
        text = body
    return text.strip()

In [2]:
pd.options.display.html.use_mathjax = False
pd.options.display.max_colwidth = 500

In [3]:
with open('../data/bloomberg-articles/news-discourse.jsonl') as f:
    all_lines = []
    for line in f:
        all_lines.append(json.loads(line))
news_discourse_flattened_df = pd.concat(list(map(pd.DataFrame, all_lines)))

In [4]:
target_beat_news_articles = pd.read_json(
    '../data/bloomberg-articles/bn-news__no-heads.json.gz',
    compression='gzip',
    lines=True,
    orient='records'
)

In [5]:
target_beat_news_articles['security_figi'] = (
    target_beat_news_articles['derived_tickers']
     .apply(lambda x: [] if isinstance(x, float) else x)
     .apply(lambda x: list(map(lambda y: y.get('security_figi'), x)))
)

In [6]:
beat_articles_discourse_labeled = (
    news_discourse_flattened_df
#         .loc[lambda df: df['discourse_preds'].isin( ['Main', 'Cause_General', 'Cause_Specific'])]
        .loc[lambda df: ~df['discourse_preds'].isin( ['NA',])]    
        .sort_values(['doc_id', 'sent_idx'])
        .groupby('doc_id')[['sentences', 'discourse_preds']]
        .aggregate(list)
        .reset_index()
)

beat_articles_discourse_labeled = (
    beat_articles_discourse_labeled
     .merge(
         target_beat_news_articles[['suid', 'timeofarrival', 'security_figi']],
         left_on='doc_id', 
         right_on='suid'
     )
     .drop(columns='doc_id')
)

In [56]:
beat_articles_discourse_labeled.to_json(
    '../data/bloomberg-articles/news-discourse-by-article-with-metadata.jsonl.gz', 
    compression='gzip',
    lines = True,
    orient='records'
)

In [7]:
doc_ids = news_discourse_flattened_df['doc_id'].drop_duplicates().tolist()

# Get prior coverage

In [8]:
import pandas as pd 
from tqdm.auto import tqdm

In [9]:
news_archive_df = []

for chunk in tqdm(pd.read_json(
    '../data/bloomberg-articles/news-archive-df.json', 
    lines=True,
    orient='records',
    chunksize=1000
)):
    news_archive_df.append(chunk)

news_archive_df = pd.concat(news_archive_df)

0it [00:00, ?it/s]

In [11]:
news_archive_df['security_figi'] = (
    news_archive_df['derived_tickers']
         .apply(lambda x: [] if isinstance(x, float) else x)
         .apply(lambda x: list(map(lambda y: y.get('security_figi'), x)))
)

news_archive_df = news_archive_df.drop(columns=['nicodes', 'tickers', 'derived_tickers'])
news_archive_df['security_figi'] = news_archive_df['security_figi'].apply(set)

In [12]:
sample_article, sample_archive = match_archive_articles(doc_ids[0])

In [13]:
a = sample_article.pipe(list_of_discourse_sents_to_df)
a

,sentence,discourse
0,"Essity AB, a Swedish maker of diapers and personal hygiene products, plans to raise prices in all markets and product areas to make up for the rising cost of raw materials, energy and distribution.",Main
1,"Those costs are ""historically high"" at the moment, and there's also nothing to indicate they will come down in the near term, the company said in a statement on Wednesday.",Distant_Evaluation
2,It had already raised prices in 2021.,Distant_Historical
3,"""Price increases are therefore essential to compensate for the higher costs,"" said Chief Executive Magnus Groth, adding that he would also be looking for efficiency measures to achieve savings.",Distant_Expectations_Consequences
4,The company reported a steeper drop in adjusted operating profit than analysts had expected for the fourth quarter of last year.,Cause_General
5,"Higher costs shaved more than 10 percentage points off of its adjusted Ebita margin, while higher selling prices and volumes helped combat some of that.",Cause_General
6,The margin fell to 9.0% from 14.2% a year earlier.,Cause_General
7,"Essity set a new target to grow sales by more than 5%, saying it will prioritize growth in categories with the highest profitability.",Distant_Expectations_Consequences


In [14]:
h = sample_archive['headline'].reset_index(drop=True)
h

0         *ESSITY ANNOUNCES PRICE INCREASES IN ALL PRODUCT CATEGORIES
1                           *ESSITY 4Q ORGANIC SALES +8%, EST. +5.22%
2                                      *ESSITY FY ADJ EBITA SEK13.68B
3     *ESSITY PRIORITIZING GROWTH IN AREAS WITH HIGHEST PROFITABILITY
4                            *ESSITY FY REV. SEK121.87B, EST. SEK121B
5          *ESSITY PRICE INCREASES IN ALL PRODUCT CATEGORIES, MARKETS
6       *ESSITY TO COMPENSATE FOR RAW MATERIALS, ENERGY, DISTRIBUTION
7                               *ESSITY FY DIV/SHR SEK7, EST. SEK6.84
8                *ESSITY: NEW TARGET FOR SALES GROWTH OF MORE THAN 5%
9                Essity 4Q Adjusted Operating Profit Misses Estimates
10                *ESSITY 4Q ADJ. OPER PROFIT SEK2.83B, EST. SEK3.40B
Name: headline, dtype: object

# Track how much of the article is retrievable

In [18]:
import requests
import os
from itertools import product
os.environ['CURL_CA_BUNDLE'] = '/Users/alex/.bb-cert/ca-bundle.trust.crt'
flan_api = 'https://flan-ul2-dev-s-ailm.inference-dev-01-pw.dsp.dev.bloomberg.com/v1/models/flan-ul2:predict'

def format_sentence_comparison(sent_a, sent_b, examples= []):    
    prompt = f"""
    Can you derive Sentence 1 from the facts expressed in Sentence 2?

    Sentence 1: {sent_a}.
    Sentence 2: {sent_b}.
    """
    return prompt
    
def hit_endpoint(sent_a, sent_b, examples=[], api_endpoint=flan_api):
    data = {
        "instances": [
            {
                "context": format_sentence_comparison(sent_a, sent_b, examples),
                "temperature": 1.1,
                "repetition_penalty": 1,
                "num_beams": 5,
            }
        ]
    }
    r = requests.post(flan_api, data=json.dumps(data))
    out = json.loads(r.text)
    return out 

In [21]:
len(a)

18

In [23]:
len(h)

356

In [24]:
18 * 356

6408

In [54]:
# sentence_comparison = []

num_retrieved = len(set(map(lambda x: x['doc_idx'], sentence_comparison)))

for doc_id in tqdm(doc_ids[num_retrieved:]):
    try:
        sample_article, sample_archive = match_archive_articles(doc_id)
        a = sample_article.pipe(list_of_discourse_sents_to_df)
        h = sample_archive['headline'].reset_index(drop=True)

        for (a_idx, a_i), (h_idx, h_i) in tqdm(
            product(enumerate(a['sentence']), enumerate(h)), total=len(a) * len(h)
        ):
            model_output = hit_endpoint(a_i, h_i)
            o = model_output['predictions'][0]['output']

            sentence_comparison.append({
                'doc_idx': doc_id, 
                'article_sentence': a_i,
                'article_sent_idx': a_idx,
                'archive_headline': h_i,
                'archive_headline_idx': h_idx,
                'model_prediction': o
            })
    except:
        continue

  0%|          | 0/1950 [00:00<?, ?it/s]

  0%|          | 0/504 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/399 [00:00<?, ?it/s]

  0%|          | 0/14112 [00:00<?, ?it/s]

...

  0%|          | 0/1113 [00:00<?, ?it/s]

  0%|          | 0/208 [00:00<?, ?it/s]

  0%|          | 0/5629 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/11374 [00:00<?, ?it/s]

In [38]:
sent_found = (pd.DataFrame(sentence_comparison)
    .groupby(['doc_idx', 'article_sent_idx'])
    .aggregate(list)
     ['model_prediction']
     .apply(lambda x: list(filter(lambda y: y.lower() == 'yes', x)))
     .pipe(lambda s: s.str.len() > 0)
)

In [39]:
sent_found.mean()

0.23684210526315788

In [36]:
d = []
for doc_id in doc_ids[:4]:
    sample_article, sample_archive = match_archive_articles(doc_id)
    a = sample_article.pipe(list_of_discourse_sents_to_df)
    d.append(a)

In [46]:
pd.concat([
    sent_found.reset_index(),
    pd.concat(d).reset_index(drop=True)['discourse']
], axis=1).groupby('discourse')['model_prediction'].mean()

discourse
Cause_General                        0.181818
Cause_Specific                            0.0
Distant_Evaluation                   0.142857
Distant_Expectations_Consequences    0.363636
Distant_Historical                        0.0
Main                                      0.5
Name: model_prediction, dtype: object

In [49]:
pd.DataFrame(sentence_comparison).loc[lambda df: df['model_prediction'].str.lower() == 'yes']

,doc_idx,article_sentence,article_sent_idx,archive_headline,archive_headline_idx,model_prediction
6,R6AZ03T0AFB4,"Essity AB, a Swedish maker of diapers and personal hygiene products, plans to raise prices in all markets and product areas to make up for the rising cost of raw materials, energy and distribution.",0,"*ESSITY TO COMPENSATE FOR RAW MATERIALS, ENERGY, DISTRIBUTION",6,Yes
39,R6AZ03T0AFB4,"""Price increases are therefore essential to compensate for the higher costs,"" said Chief Executive Magnus Groth, adding that he would also be looking for efficiency measures to achieve savings.",3,"*ESSITY TO COMPENSATE FOR RAW MATERIALS, ENERGY, DISTRIBUTION",6,Yes
53,R6AZ03T0AFB4,The company reported a steeper drop in adjusted operating profit than analysts had expected for the fourth quarter of last year.,4,Essity 4Q Adjusted Operating Profit Misses Estimates,9,Yes
54,R6AZ03T0AFB4,The company reported a steeper drop in adjusted operating profit than analysts had expected for the fourth quarter of last year.,4,"*ESSITY 4Q ADJ. OPER PROFIT SEK2.83B, EST. SEK3.40B",10,Yes
80,R6AZ03T0AFB4,"Essity set a new target to grow sales by more than 5%, saying it will prioritize growth in categories with the highest profitability.",7,*ESSITY PRIORITIZING GROWTH IN AREAS WITH HIGHEST PROFITABILITY,3,Yes
109,R7MGYFT0AFB4,"McDonald's says it will evaluate Carl Icahn's board nominees, Leslie Samuelrich and Maisie Ganzler, as it would any other candidates. """,0,*MCDONALD'S TO EVALUATE ICAHN NOMINEES AS IT WOULD ANY OTHERS,21,Yes
271,R9E46BDWRGG1,Saudi Arabian grocery chain BinDawood Holding bought a majority stake in e-commerce app manager and developer International Applications Trading Co. for an initial payment of 107.5 million riyals ($28.7 million) that could increase subject to future profitability milestones.,1,Bindawood Unit Signs Pact to Buy Stake in IATC For 107.5m Riyals,1,yes
278,R9E46BDWRGG1,"BinDawood Holding, through its Future Technology Retail unit, signed a definitive agreement to acquire a 62% stake in IATC, according to a statement.",2,*BINDAWOOD UNIT IN MOU TO BUY 62% OF INTL APPLICATIONS TRADING,0,Yes
281,R9E46BDWRGG1,"BinDawood Holding, through its Future Technology Retail unit, signed a definitive agreement to acquire a 62% stake in IATC, according to a statement.",2,*BINDAWOOD UNIT SIGN PACT TO BUY SHARES IN IATC,3,Yes
283,R9E46BDWRGG1,"BinDawood Holding, through its Future Technology Retail unit, signed a definitive agreement to acquire a 62% stake in IATC, according to a statement.",2,BinDawood Unit Signs MoU to Buy 62% of Intl Applications Trading,5,yes


# Example of Document with linguistic differences 

In [141]:
doc_id = 'R9NDFIT0AFB5'
sample_article, sample_archive = match_archive_articles(doc_id)

In [140]:
# (beat_articles_discourse_labeled
#  .loc[lambda df: df['sentences'].str.join(' ').str.contains('Aldar') ]
# )

In [146]:
sample_article.pipe(list_of_discourse_sents_to_df)
# a lot of bloomberg stories look at past earnings (MA function in bloomberg)
# a possible angle for this story would have been looking at the owner and his wealth

# would be good to know if these conclusions generalize to other types of stories within bloomberg
# go back and find some beat reporting that is not part of these bundles

,sentence,discourse
0,"Aldar Properties PJSC acquired a beach resort for $210 million in the United Arab Emirates, the latest move by the real-estate developer to expand outside its home base of Abu Dhabi.",Main
1,"The company bought the 715-room Rixos Bab Al Bahr beach property in Ras Al Khaimah with additional development rights, according to a statement.",Cause_General
2,The transaction takes Aldar's total investment in Ras Al Khaimah to 1.5 billion dirhams ($408 million).,Cause_General
3,Ras Al Khaimah and Abu Dhabi are among seven emirates that make up the UAE.,Cause_General
4,"At home, the developer is growing its asset management business and beyond the UAE, it's gained a foothold in Egypt through its acquisition of a majority stake in a developer known as Sodic.",Cause_General
5,"Apollo's $1.4 Billion Abu Dhabi Deal Marks Latest Gulf Foray Apollo Global Management Inc. in February signed a deal to invest $1.4 billion in Aldar, in what could be one of the largest foreign direct investments in Abu Dhabi's private sector.",Cause_General
6,Aldar's shares have gained 22% this year compared with a 17% gain for the benchmark Abu Dhabi stock index.,Cause_General


In [147]:
sample_archive['headline']

1141                                         Intrado Buys Permission Click
1199                      *ALDAR FY PROFIT REACHED AED2.33B, UP BY 21% Y/Y
1221                   *TENNECO WILL BECOME A PRIVATE CO. AFTER DEAL CLOSE
1805           *PEARSON: BOARD UNANIMOUSLY REJECTED SECOND APOLLO PROPOSAL
3464               *ALDAR TO INVEST FUNDS INTO EDUCATION UNIT IN ABU DHABI
                                        ...                               
108667                                  Intrado to Sell Flowroute Business
111586    *APOLLO TARGETS $50B IN CLEAN ENERGY, CLIMATE CAPITAL NEXT 5 YRS
112663      Alpha Dhabi Buys $1.5 Billion Stake in UAE Property Firm Aldar
114514       *APOLLO ANNOUNCES FIRST CLOSE OF GLOBAL WEALTH ACQUISITION OF
115097     *APOLLO TO INVEST IN ABU DHABI'S ALDAR PROPERTIES IN $1.4B DEAL
Name: headline, Length: 130, dtype: object